In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['C:\\Users\\troy\\Documents\\Code\\Libraries', 'C:\\Users\\troy\\Documents\\Excel Sheets', 'C:/Users/troy/Documents/Code/Libraries'])
from datetime import date, datetime
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import autoreload

import dash_bootstrap_components as dbc
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output

from FinAPI.PortfolioAnalysis.AnalyzePortfolio import Portfolio


Python 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)] on win32


In [2]:
today = date(2021, 9, 24)
objPortfolio = Portfolio(today)
trader_names = objPortfolio.trader_names(objPortfolio.raw_data)

Notional
Secondary Notional
Currency1 Notional
Currency2 Notional
Trade Price
Accrued Interest


In [3]:
# Components
trade_df = objPortfolio.raw_data

positions_table = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in trade_df.columns],
    data=trade_df.to_dict('records'),
    filter_action='native',
    style_cell={'textAlign': 'center', 'maxWidth': '100px'},
    style_header={
        'backgroundColor': 'gray',
        'fontWeight': 'bold', 
        'textAlign': 'center'
    },
    style_data={
        'whiteSpace': 'normal',
        'height': 'auto',
    },
    style_data_conditional=[
        {
            'if': {
                'filter_query': '{Trade Name} != ""',
            },
            'borderTop': '2px rgb(100, 100, 100) solid',
        },
        {
            'if': {
                'filter_query': '{MCMM} < 0',
                'column_id': 'MCMM'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{MassPRIM} < 0',
                'column_id': 'MassPRIM'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{PSAMF} < 0',
                'column_id': 'PSAMF'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{RVMaster} < 0',
                'column_id': 'RVMaster'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{UCITS} < 0',
                'column_id': 'UCITS'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{Total} < 0',
                'column_id': 'Total'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {'column_id': 'Trade Name'},
            'textAlign': 'left',
            'maxWidth': '150px'
        },
        {
            'if': {'column_id': 'Security Id'},
            'textAlign': 'left',
            'maxWidth': '150px'
        }
    ]
    )

dropdown = dcc.Dropdown(id="trader_name_filter",options=[{'label':tname, 'value':tname} for tname in trader_names],value='troy')

table_filters = dcc.Checklist(
    id='trade_table_checklist',
    options=[
        {'label': 'Remove Closed Trades', 'value': 'NoClosed'},
        {'label': 'Remove Matured ISINS', 'value': 'NoExpired'},
    ],
    value=['NoClosed', 'NoExpired'],
    labelStyle={'display': 'inline-block'}
)

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Portfolio Dashboard (Data: %s)" % today.strftime('%d-%m-%Y')),
    dropdown,
    table_filters,
    positions_table
])

# Run app and display result inline in the notebook
app.run_server(mode='inline')
